In [23]:
#FULL CODE

from tkinter import *
import tkinter as tk
from tkinter import ttk
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
import requests
import random
import time

#Required credentials
#Spotify Developer credentials from developers.spotify.com
client_id = '2920ec1af7074a2f96e859b0246257f3'
client_secret = 'c986a95cc7f54a9a869ca9969eff9459'
redirect_uri = 'http://localhost:8888/callback/' #Not using this though 

#Authenticate with Spotify
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

custom_playlist = []

class main:
    def __init__(self, root):
        self.root = root
        self.root.title("Custom Music Playlist")
        self.root.geometry("1000x1000")
        
        self.canvas = Canvas(self.root, width=800, height=250, bg="khaki1")
        self.canvas.pack()
        
        self.display = self.canvas.create_text(415, 125, text="", fill="black", font=('Helvetica 15 bold'))
        self.display2 = self.canvas.create_text(415, 215, text="", fill="black", font=('Helvetica 15 bold'))
        
        self.feedback = Label(root, text="", pady=10, font=("arial", 15, "bold"))
        
        self.intro_blurb()

    def intro_blurb(self):
        intro = f'''Welcome to our music program! This is a fun program that will build you a custom playlist based on YOUR interests
        and at no cost! You will be able to look into different music reccomendations based on year, genre, etc. and if you would 
        like add them to your custom playlist! Our hope is that this not only allows you to create a playlist to always enjoy
        jamming to, but that it may also introduce you to new music of different styles, cultures, and backgrounds that you
        may carry with you for many years. We hope you enjoy!'''
        self.canvas.itemconfig(self.display, text=intro.strip(), width=750)

        self.next_page = ttk.Button(root, text="Next Page", command = self.song_of_the_day)
        self.next_page.pack()


    def song_of_the_day(self):
        songofday_list = []

        song_of_day_intro = f'''Our song of the day is a current top hit, that has been taking over the music recently. Many
        people have been enjoying these songs, and we hope we can introduce you to one as well!'''

        self.canvas.itemconfig(self.display, text=song_of_day_intro, width=500)

        for i in range(50):
            song_of_day_playlist = sp.playlist("37i9dQZF1DXcBWIGoYBM5M")
            song = song_of_day_playlist["tracks"]["items"][i]["track"]["name"]
            songofday_list.append(song)

        random_song = random.choices(songofday_list,k=1)

        self.canvas.itemconfig(self.display, 
                          text=f"Today's song of the day is {random_song[0]}! We hope you give a listen and enjoy!",
                          width=500)

        song_of_day = f"Would you like to add {random_song[0]} to your playlist?"
        self.canvas.itemconfig(self.display2, text=song_of_day, width=500)

        def add_playlist():
            custom_playlist.append(random_song[0])
            self.feedback["text"] = "Song added!"
            self.feedback.pack()

        def no_add():
            pass

        self.yes_button = Button(root, text="Yes", command=add_playlist, width=25, height=5)
        self.yes_button.pack()

        self.no_button = Button(root, text="No", command=no_add, width=25, height=5)
        self.no_button.pack()
        
        self.next_page1 = ttk.Button(root, text="Next Page", command = self.rec_song_by_artist)
        self.next_page1.pack()
        
    def rec_song_by_artist(self):    
        artist_chosen = f"Please enter an artist you enjoy, and we will reccomend a song based on a similar artist:"
        self.canvas.itemconfig(self.display, text=artist_chosen)

        self.entry = Entry(root)
        self.entry.pack()

        reccomended_song = []

        def search_rec_artist():
            user_input = self.entry.get()

            results1 = sp.search(q=user_input, type='artist', limit=1)

            for artist in results1['artists']['items']:
                artist_id1 = artist['id']

            results = sp.recommendations(seed_artists=[artist_id1])
            rec_art = results["tracks"][3]['album']['artists'][0]['name']

            results2 = sp.search(q=rec_art, type='artist', limit=1)
            for k in results2['artists']['items']:
                artist_id2 = k['id']

            rec_songs = sp.artist_top_tracks(artist_id2)
            rec_song = rec_songs["tracks"][0]["name"]

            reccomended_song.append(rec_song)

            song_of_add1 = f"Would you like to add {rec_song}, by {rec_art} to your playlist?"
            self.canvas.itemconfig(self.display2, text=song_of_add1)

        self.search_button = Button(root, text="Search", command=search_rec_artist)
        self.search_button.pack()

        def add_playlist1():
            custom_playlist.append(reccomended_song[0])
            self.feedback["text"] = f"Song Added: {reccomended_song[0]}"
            self.feedback.pack()

        def no_add1():
            pass

        self.yes_button1 = Button(root, text="Yes", command=add_playlist1, width=25, height=5)
        self.yes_button1.pack()

        self.no_button1 = Button(root, text="No", command=no_add1, width=25, height=5)
        self.no_button1.pack()
        
        self.next_page2 = ttk.Button(root, text="Next Page", command = self.rec_song_by_genre)
        self.next_page2.pack()
        
    def rec_song_by_genre(self):
        genre_choice = "Please choose a genre from the list, and we will reccomend a song based on that genre"
        self.canvas.itemconfig(self.display, text=genre_choice)

        self.entry = Entry(root)
        self.entry.pack()

        genre_song = []

        def search_genre():
            user_choice = self.entry.get()

            rec1 = sp.recommendations(seed_genres=[user_choice], limit=10)
            rec_song_by_genre = rec1["tracks"][0]["name"]
            rec_art_by_genre = rec1["tracks"][0]["artists"][0]['name']
            genre_song.append(rec_song_by_genre)

            song_of_add2 = f"Would you like to add {rec_song_by_genre}, by {rec_art_by_genre} to your playlist? Type YES to add it!"
            self.canvas.itemconfig(self.display2, text=song_of_add2, width=500)

        self.search_button1 = Button(root, text="Search", command=search_genre)
        self.search_button1.pack()

        def add_playlist2():
            custom_playlist.append(genre_song[0])
            self.feedback["text"] = f"Song Added:{genre_song[0]}"
            self.feedback.pack()

        def no_add2():
            pass

        self.yes_button2 = Button(root, text="Yes", command=add_playlist2, width=25, height=5)
        self.yes_button2.pack()

        self.no_button2 = Button(root, text="No", command=no_add2, width=25, height=5)
        self.no_button2.pack()

        self.next_page3 = ttk.Button(root, text="Next Page", command = self.rec_song_by_album)
        self.next_page3.pack()
    
    def rec_song_by_album(self):
        album_chosen = "All playlists need some familiarity, so please enter your favorite album, and we will choose a song from that album:"
        self.canvas.itemconfig(self.display, text=album_chosen)

        self.entry = Entry(root)
        self.entry.pack()

        random_album_song = []

        def search_album():
            album_choice = self.entry.get()

            results2 = sp.search(q=album_choice, type='album', limit=1)
            for album in results2['albums']['items']:
                album_id1 = album['id']

            alb_info = sp.album(album_id1)
            songs_from_album = alb_info['tracks']['items']

            random_songs_from_album = []

            for i in songs_from_album:
                album_song = i['name']
                random_songs_from_album.append(album_song)

            random_choice_alb = random.choice(random_songs_from_album)
            random_album_song.append(random_choice_alb)

            song_of_add3 = f"Would you like to add {random_choice_alb} to your playlist? Type YES to add it!"
            self.canvas.itemconfig(self.display2, text=song_of_add3, width=500)

        self.search_button2 = Button(root, text="Search", command=search_album)
        self.search_button2.pack()

        def add_playlist3():
            custom_playlist.append(random_album_song[0])
            self.feedback["text"] = f"Song Added:{random_album_song[0]}"
            self.feedback.pack()

        def no_add3():
            pass

        self.yes_button3 = Button(root, text="Yes", command=add_playlist3, width=25, height=5)
        self.yes_button3.pack()

        no_button3 = Button(root, text="No", command=no_add3, width=25, height=5)
        no_button3.pack()
        
        self.next_page4 = ttk.Button(root, text="Next Page", command = self.rec_song_by_decade)
        self.next_page4.pack()
        
        
    def rec_song_by_decade(self):    
        decade = "Please enter your FAVORITE decade for music, and we will reccomend one of the top hits from that decade:" 
        self.canvas.itemconfig(self.display, text=decade)

        self.entry = Entry(root)
        self.entry.pack()

        decades_throwbacks = []

        def search_decade():
            decade_chosen = self.entry.get()

            dec_results = sp.search(q = decade_chosen, type= 'playlist', limit= 1)
            playlist_id = dec_results['playlists']['items'][0]['id']

            dec_results1 = sp.playlist(playlist_id)
            dec_results2 = dec_results1['tracks']['items']

            throwbacks = []

            for i in dec_results2:
                dec_song = i['track']['name']
                throwbacks.append(dec_song)

            random_throwback = random.choice(throwbacks)
            decades_throwbacks.append(random_throwback)

            song_of_add4 = f"Would you like to add {random_throwback} to your playlist? Type YES to add it!"
            self.canvas.itemconfig(self.display2, text=song_of_add4, width=500)

        self.search_button3 = Button(root, text="Search", command=search_decade)
        self.search_button3.pack()

        def add_playlist4():
            custom_playlist.append(decades_throwbacks[0])
            self.feedback["text"] = f"Song Added:{decades_throwbacks[0]}"
            self.feedback.pack()

        def no_add4():
            pass

        self.yes_button = Button(root, text="Yes", command=add_playlist4, width=25, height=5)
        self.yes_button.pack()

        self.no_button = Button(root, text="No", command=no_add4, width=25, height=5)
        self.no_button.pack()

root = Tk()
main(root)
root.mainloop()

In [22]:
#song of the day page

root = Tk()
root.title("Custom Music Playlist")
root.geometry("1000x1000")

canvas = Canvas(root, width=800, height=250, bg="khaki1")
canvas.pack()

display = canvas.create_text(415, 125, text="", fill="black", font=('Helvetica 15 bold'))
display2 = canvas.create_text(415, 215, text="", fill="black", font=('Helvetica 15 bold'))

feedback = Label(root, text="", pady=10, font=("arial", 15, "bold"))

def song_of_the_day():
    songofday_list = []

    song_of_day_intro = f'''Our song of the day is a current top hit, that has been taking over the music recently. Many
    people have been enjoying these songs, and we hope we can introduce you to one as well!'''

    canvas.itemconfig(display, text=song_of_day_intro, width=500)

    for i in range(50):
        song_of_day_playlist = sp.playlist("37i9dQZF1DXcBWIGoYBM5M")
        song = song_of_day_playlist["tracks"]["items"][i]["track"]["name"]
        songofday_list.append(song)

    random_song = random.choices(songofday_list,k=1)

    canvas.itemconfig(display, 
                      text=f"Today's song of the day is {random_song[0]}! We hope you give a listen and enjoy!",
                      width=500)

    song_of_day = f"Would you like to add {random_song[0]} to your playlist?"
    canvas.itemconfig(display2, text=song_of_day, width=500)

    def add_playlist():
        custom_playlist.append(random_song[0])
        feedback["text"] = "Song added!"
        feedback.pack()

    def no_add():
        pass

    yes_button = Button(root, text="Yes", command=add_playlist, width=25, height=5)
    yes_button.pack()

    no_button = Button(root, text="No", command=no_add, width=25, height=5)
    no_button.pack()
        
    next_page1 = ttk.Button(root, text="Next Page", command = rec_song_by_artist)
    next_page1.pack()

song_of_the_day()
root.mainloop()

In [13]:
#reccomend song by artist page

root = Tk()
root.title("Custom Music Playlist")
root.geometry("1000x1000")

canvas = Canvas(root, width=800, height=250, bg="khaki1")
canvas.pack()

display = canvas.create_text(415, 125, text="", fill="black", font=('Helvetica 15 bold'))
display2 = canvas.create_text(415, 215, text="", fill="black", font=('Helvetica 15 bold'))

feedback = Label(root, text="", pady=10, font=("arial", 15, "bold"))

def rec_song_by_artist():    
    artist_chosen = f"Please enter an artist you enjoy, and we will reccomend a song based on a similar artist:"
    canvas.itemconfig(display, text=artist_chosen)
    
    entry = Entry(root)
    entry.pack()
    
    reccomended_song = []
    
    def search_rec_artist():
        user_input = entry.get()
        
        results1 = sp.search(q=user_input, type='artist', limit=1)

        for artist in results1['artists']['items']:
            artist_id1 = artist['id']

        results = sp.recommendations(seed_artists=[artist_id1])
        rec_art = results["tracks"][3]['album']['artists'][0]['name']

        results2 = sp.search(q=rec_art, type='artist', limit=1)
        for k in results2['artists']['items']:
            artist_id2 = k['id']

        rec_songs = sp.artist_top_tracks(artist_id2)
        rec_song = rec_songs["tracks"][0]["name"]
        
        reccomended_song.append(rec_song)

        song_of_add1 = f"Would you like to add {rec_song}, by {rec_art} to your playlist?"
        canvas.itemconfig(display2, text=song_of_add1)
    
    search_button = Button(root, text="Search", command=search_rec_artist)
    search_button.pack()
    
    def add_playlist():
        custom_playlist.append(reccomended_song[0])
        feedback["text"] = f"Song Added: {reccomended_song[0]}"
        feedback.pack()

    def no_add():
        pass
    
    yes_button = Button(root, text="Yes", command=add_playlist, width=25, height=5)
    yes_button.pack()

    no_button = Button(root, text="No", command=no_add, width=25, height=5)
    no_button.pack()
        
rec_song_by_artist()
root.mainloop()

In [14]:
#recommend song by genre page

root = Tk()
root.title("Custom Music Playlist")
root.geometry("1000x1000")

canvas = Canvas(root, width=800, height=250, bg="khaki1")
canvas.pack()

display = canvas.create_text(415, 125, text="", fill="black", font=('Helvetica 15 bold'))
display2 = canvas.create_text(415, 215, text="", fill="black", font=('Helvetica 15 bold'))

feedback = Label(root, text="", pady=10, font=("arial", 15, "bold"))

genres = sp.recommendation_genre_seeds()

def rec_song_by_genre():
    genre_choice = "Please choose a genre from the list, and we will reccomend a song based on that genre"
    canvas.itemconfig(display, text=genre_choice)
    
    entry = Entry(root)
    entry.pack()
    
    genre_song = []
    
    def search_genre():
        user_choice = entry.get()
        
        rec1 = sp.recommendations(seed_genres=[user_choice], limit=10)
        rec_song_by_genre = rec1["tracks"][0]["name"]
        rec_art_by_genre = rec1["tracks"][0]["artists"][0]['name']
        genre_song.append(rec_song_by_genre)

        song_of_add2 = f"Would you like to add {rec_song_by_genre}, by {rec_art_by_genre} to your playlist? Type YES to add it!"
        canvas.itemconfig(display2, text=song_of_add2, width=500)

    search_button = Button(root, text="Search", command=search_genre)
    search_button.pack()
    
    def add_playlist():
        custom_playlist.append(genre_song[0])
        feedback["text"] = f"Song Added: {genre_song[0]}"
        feedback.pack()

    def no_add():
        pass
    
    yes_button = Button(root, text="Yes", command=add_playlist, width=25, height=5)
    yes_button.pack()

    no_button = Button(root, text="No", command=no_add, width=25, height=5)
    no_button.pack()

rec_song_by_genre()
root.mainloop()

In [15]:
#reccomend song by album page

root = Tk()
root.title("Custom Music Playlist")
root.geometry("1000x1000")

canvas = Canvas(root, width=800, height=250, bg="khaki1")
canvas.pack()

display = canvas.create_text(415, 125, text="", fill="black", font=('Helvetica 15 bold'))
display2 = canvas.create_text(415, 215, text="", fill="black", font=('Helvetica 15 bold'))

feedback = Label(root, text="", pady=10, font=("arial", 15, "bold"))

def rec_song_by_album():
    album_chosen = "All playlists need some familiarity, so please enter your favorite album, and we will choose a song from that album:"
    canvas.itemconfig(display, text=album_chosen)

    entry = Entry(root)
    entry.pack()
    
    random_album_song = []
    
    def search_album():
        album_choice = entry.get()
        
        results2 = sp.search(q=album_choice, type='album', limit=1)
        for album in results2['albums']['items']:
            album_id1 = album['id']

        alb_info = sp.album(album_id1)
        songs_from_album = alb_info['tracks']['items']

        random_songs_from_album = []

        for i in songs_from_album:
            album_song = i['name']
            random_songs_from_album.append(album_song)

        random_choice_alb = random.choice(random_songs_from_album)
        random_album_song.append(random_choice_alb)
        
        song_of_add3 = f"Would you like to add {random_choice_alb} to your playlist? Type YES to add it!"
        canvas.itemconfig(display2, text=song_of_add3, width=500)

    search_button = Button(root, text="Search", command=search_album)
    search_button.pack()
    
    def add_playlist():
        custom_playlist.append(random_album_song[0])
        feedback["text"] = f"Song Added: {random_album_song[0]}"
        feedback.pack()
    
    def no_add():
        pass
    
    yes_button = Button(root, text="Yes", command=add_playlist, width=25, height=5)
    yes_button.pack()

    no_button = Button(root, text="No", command=no_add, width=25, height=5)
    no_button.pack()

rec_song_by_album()
root.mainloop()

In [16]:
#reccomend song by decade page

root = Tk()
root.title("Custom Music Playlist")
root.geometry("1000x1000")

canvas = Canvas(root, width=800, height=250, bg="khaki1")
canvas.pack()

display = canvas.create_text(415, 125, text="", fill="black", font=('Helvetica 15 bold'))
display2 = canvas.create_text(415, 215, text="", fill="black", font=('Helvetica 15 bold'))

feedback = Label(root, text="", pady=10, font=("arial", 15, "bold"))

def rec_song_by_decade():    
    decade = "Please enter your FAVORITE decade for music, and we will reccomend one of the top hits from that decade:" 
    canvas.itemconfig(display, text=decade)
    
    entry = Entry(root)
    entry.pack()
    
    decades_throwbacks = []
    
    def search_decade():
        decade_chosen = entry.get()
        
        dec_results = sp.search(q = decade_chosen, type= 'playlist', limit= 1)
        playlist_id = dec_results['playlists']['items'][0]['id']

        dec_results1 = sp.playlist(playlist_id)
        dec_results2 = dec_results1['tracks']['items']

        throwbacks = []

        for i in dec_results2:
            dec_song = i['track']['name']
            throwbacks.append(dec_song)

        random_throwback = random.choice(throwbacks)
        decades_throwbacks.append(random_throwback)
        
        song_of_add4 = f"Would you like to add {random_throwback} to your playlist? Type YES to add it!"
        canvas.itemconfig(display2, text=song_of_add4, width=500)
        
    search_button = Button(root, text="Search", command=search_decade)
    search_button.pack()
    
    def add_playlist():
        custom_playlist.append(decades_throwbacks[0])
        feedback["text"] = f"Song Added: {decades_throwbacks[0]}"
        feedback.pack()
    
    def no_add():
        pass
    
    yes_button = Button(root, text="Yes", command=add_playlist, width=25, height=5)
    yes_button.pack()

    no_button = Button(root, text="No", command=no_add, width=25, height=5)
    no_button.pack()
    
rec_song_by_decade()
root.mainloop()